# Preparation
Aqui o script instala o ferramental necessário para o experimento.


1.   Acesso a api-key do grok (foi a llm utilizada no vídeo)
2.   Instalação da dependência da api do groq. (Ela utilizará a api-key pra se comunicar com os servidores do grok)



In [23]:
from google.colab import userdata
llm_api_key = userdata.get('grok_api_key')

!pip install groq

# Teste de configuração
Aqui é feito somente uma interação com o grok pra saber se está ok. Em resumo testa a dependência e a api-key.

In [24]:
import os

from groq import Groq

client = Groq(
    api_key = llm_api_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in modern natural language processing (NLP) applications due to their ability to process and generate human-like language quickly and efficiently. The importance of fast language models can be seen in several areas:

1. **Improved User Experience**: Fast language models enable real-time interactions, allowing users to receive immediate responses to their queries, commands, or requests. This is particularly important for applications like chatbots, virtual assistants, and conversational interfaces, where users expect rapid and seamless interactions.
2. **Increased Productivity**: Fast language models can automate tasks, such as text summarization, translation, and content generation, freeing up human time and resources for more complex and creative tasks. This can significantly boost productivity in industries like customer service, content creation, and language translation.
3. **Enhanced Decision-Making**: Fast language models can quickly analyze large

# Implementação dos pilares (agente e system prompt)

## Definição da classe do Agente
O agente é definido com a estrutura de construção, inicializando seu estado, e uma function __call__ para registrar as entradas na pilha de interação/conversa. Adicionalmente, há um método execute que dispara a chamada com pilha de conversas para a api de chat completion da llm, no caso o grok.

In [25]:
class Agent:

  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})


  def __call__(self, message = ""):
    if message is not None:
      self.messages.append({"role": "user", "content": message})

    result = self.execute()

    self.messages.append({"role": "assistant", "content": result})
    return result


  def execute(self):
    completion = self.client.chat.completions.create(
      messages=self.messages,
      model="llama-3.3-70b-versatile",
    )
    return completion.choices[0].message.content

## Definição do prompt
Dentro da stack de mensagens do agente é a primeira mensagem. É onde estão descritas as características basilares do agente.
Nesse exemplo, é a definição comportamental diante das interações, e a descrição das ferramentas que estarão disponíveis. O exemplo no final é para reforçar como seria a dinâmica da comunicação entre o cliente e o agente.

In [26]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [27]:
# Tools

def calculate (operation):
  return eval(operation)


def get_planet_mass(planet) -> float:

  planet = planet.lower()

  if planet == "earth":
    return 5.972e24
  elif planet == "mars":
    return 0.642e24
  elif planet == "jupiter":
    return 1.899e27
  elif planet == "saturn":
    return 5.688e26
  elif planet == "uranus":
    return 8.686e25
  elif planet == "neptune":
    return 1.024e26
  elif planet == "mercury":
    return 0.33e24
  elif planet == "venus":
    return 4.87e24
  else:
    return 0.0


# Implementação da interação programaticamente
O fluxo está sendo controlado pelo programador, mimificando o comportamento desejado do agente.

In [28]:
neil_tyson = Agent(client, system_prompt)

neil_tyson.messages


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

In [29]:
result = neil_tyson("What is the mass of the earth multiplied by 5?")

print(result)

Thought: To find the mass of the Earth multiplied by 5, I first need to find the mass of the Earth. 
Action: get_planet_mass: Earth
PAUSE


In [30]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

In [31]:
observation = get_planet_mass("earth")

next_message = "Observation: " + str(observation)

result = neil_tyson(next_message)

print(result)

Thought: Now that I have the mass of the Earth, I can multiply it by 5 to get the final answer.
Action: calculate: 5.972e+24 * 5
PAUSE


In [32]:
observation = calculate("5.972e+24 * 5")

next_message = f"Observation: {observation}"

result = neil_tyson(next_message)

print(result)

Thought: I have now calculated the mass of the Earth multiplied by 5, so I can output the final answer.
Answer: The mass of the Earth multiplied by 5 is 2.986e+25.


In [33]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

# Definição do loop de interação
Aqui um loop foi definido para manter a interação da llm e ferramentas locais, fazendo as chamadas necessárias até o loop ser encerrado.

In [34]:
import re

# Loop do agente. Cada iteração, a stack message (contexto) é enriquecido com
# uma mensagem de interação para o llm e sua resposta. Se sua resposta
# solicitar uso de ferramentas (tools), eles são executados e suas respostas,
# acrescidas ao contexto para a próxima iteração. Isso se repete até uma
# reposta ser encontrada ou estourar o número máximo de iterações.
def agent_loop(max_iterations = 10, query = ""):
  agent = Agent(client, system_prompt)
  tools = ["calculate", "get_planet_mass"]
  next_prompt = query
  i = 0
  while i < max_iterations:
      i += 1
      result = agent(next_prompt)
      print(result)

      if "PAUSE" in result and "Action" in result:
          action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
          chosen_tool = action[0][0]
          arg = action[0][1]

          if chosen_tool in tools:
              result_tool = eval(f"{chosen_tool}('{arg}')")
              next_prompt = f"Observation: {result_tool}"

          else:
              next_prompt = "Observation: Tool not found"

          print(next_prompt)
          continue

      if "Answer" in result:
          break


agent_loop(query="What is the mass of Earth times 2?")


Thought: I need to find the mass of Earth and then multiply it by 2 to get the result. The first step is to get the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I can proceed to multiply it by 2 to find the result.

Action: calculate: 5.972e+24 * 2
PAUSE
Observation: 1.1944e+25
Thought: I have now calculated the mass of Earth times 2, so I can provide the final answer.

Answer: The mass of Earth times 2 is 1.1944e+25.
